In [1]:
# We import the appropriate libraries
import osmnx as ox, networkx as nx, geopandas as gpd, matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, mapping
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
import pyproj
import pandas as pd
import json

In [2]:
# We define the function that will calculate the isochrones for each minute on the prip time range and other criteria, translate
# the resulting shapes into lat long from UTM, convert them to geojson and save the code on a dataframe along with descriptive
# information for each shape
def geojson_isochrones_luxembourg_trainstations(dataframe, network_type='walk', trip_times_range=31, travel_speed=4.5, distance=2500):
    # We create a list of trip times from 1 to trip_times_range - 1 to iterate over
    trip_times = [i for i in range(trip_times_range) if i > 0]
    # We create an empty dataframe that once filled will be returned by the function, with name of the station, network type ('walk'
    # or 'bike'), distance refering to size of map rendered during calculations, travel speed, trip time in minutes and the geojson
    # shape
    dfpolygons = pd.DataFrame(columns=['name', 'network_type', 'distance', 'speed', 'trip_time', 'geojson'])
    # Initiate the index that will be used to fill the previous dataframe
    polyindex = 0
    # We iterate over the dataframe that has the list of names and locations of train stations
    for index, row in dataframe.iterrows():
        station = (dataframe['lat'][index], dataframe['long'][index])
        # download the street network
        G = ox.graph_from_point(station, distance=distance, network_type=network_type)
        # find the centermost node and then project the graph to UTM
        gdf_nodes = ox.graph_to_gdfs(G, edges=False)
        x, y = gdf_nodes['geometry'].unary_union.centroid.xy
        center_node = ox.get_nearest_node(G, (y[0], x[0]))
        G = ox.project_graph(G)
        # add an edge attribute for time in minutes required to traverse each edge
        meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
        # we set the weights of the edges to travel time in minutes
        for u, v, k, data in G.edges(data=True, keys=True):
            data['time'] = data['length'] / meters_per_minute
        # We iterate of the trip times to calculate the isochrone for each trip time
        for trip_time in sorted(trip_times, reverse=True):
            subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
            node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
            # We calculate the isochrone
            bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
            # We use pyproj to create a translator from UTM to lat long, as illustrated by user richard in https://stackoverflow.com/questions/6778288/lat-lon-to-utm-to-lat-lon-is-extremely-flawed-how-come
            p = pyproj.Proj(proj='utm', zone=32, ellps='WGS84')
            # We only translate and save the polygonal isochrone shapes discarding line and point shapes which would be hard to
            # interpret on a map
            if 'Polygon' in gpd.GeoSeries([bounding_poly]).to_json():                    
                x,y = mapping(bounding_poly)['coordinates'][0][0]
                p(x,y,inverse=True)
                # we translate 
                new_mapping = []
                for i in mapping(bounding_poly)['coordinates'][0]:
                    x,y = i
                    new_mapping.append(list(p(x,y,inverse=True)))
                # We fit the new mappings on the geometry of a geojson polygon shape
                new_geo_json = {"type": "FeatureCollection","features": [{"type": "Feature","geometry": {"type": "Polygon","coordinates": [new_mapping]},"properties": {}}]}
                # We save the translated isochrone with its parameters in the dataframe
                dfpolygons.loc[polyindex] = [dataframe['name'][index], network_type, distance, travel_speed, trip_time, new_geo_json]
                polyindex = polyindex + 1
    return dfpolygons

In [3]:
# We download the dataframe with the station coordinates
dataframe = pd.read_csv('Data\luxembourg_rail_coordinates.csv')

In [4]:
# We configure a dataframe of isochrones with the standard configuration
walk_2500_4point5_30 = geojson_isochrones_luxembourg_trainstations(dataframe)

In [5]:
walk_2500_4point5_30.head()

,name,network_type,distance,speed,trip_time,geojson
0,Luxembourg Gare Centrale,walk,2500,4.5,30,"{'type': 'FeatureCollection', 'features': [{'t..."
1,Luxembourg Gare Centrale,walk,2500,4.5,29,"{'type': 'FeatureCollection', 'features': [{'t..."
2,Luxembourg Gare Centrale,walk,2500,4.5,28,"{'type': 'FeatureCollection', 'features': [{'t..."
3,Luxembourg Gare Centrale,walk,2500,4.5,27,"{'type': 'FeatureCollection', 'features': [{'t..."
4,Luxembourg Gare Centrale,walk,2500,4.5,26,"{'type': 'FeatureCollection', 'features': [{'t..."


In [6]:
# We save the dataframe
walk_2500_4point5_30.to_csv('Data\luxembourg_trainstations_isochrones_walk_2500_4-5_30.csv',index='geojson')

In [7]:
# We configure a dataframe of isochrones for cycling transportation moving at twice the speed thus we meed to render maps twice
# the distance meaning 4 times the area for the calculations of isochrones
bike_5000_9_30 = geojson_isochrones_luxembourg_trainstations(dataframe, network_type='bike', travel_speed=9, distance=5000)

In [8]:
bike_5000_9_30.head()

,name,network_type,distance,speed,trip_time,geojson
0,Luxembourg Gare Centrale,bike,5000,9,30,"{'type': 'FeatureCollection', 'features': [{'t..."
1,Luxembourg Gare Centrale,bike,5000,9,29,"{'type': 'FeatureCollection', 'features': [{'t..."
2,Luxembourg Gare Centrale,bike,5000,9,28,"{'type': 'FeatureCollection', 'features': [{'t..."
3,Luxembourg Gare Centrale,bike,5000,9,27,"{'type': 'FeatureCollection', 'features': [{'t..."
4,Luxembourg Gare Centrale,bike,5000,9,26,"{'type': 'FeatureCollection', 'features': [{'t..."


In [9]:
bike_5000_9_30.to_csv('Data\luxembourg_trainstations_isochrones_bike_5000_9_30.csv',index='geojson')

In [10]:
# We select and configure a geojson for representation in folium as would be done when configuring the final selection of isochrones
# to display, in this example it is an isochrone for the area 5 minutes walking + train distance from the initial station
geojson_for_representation = walk_2500_4point5_30.loc[walk_2500_4point5_30['trip_time'] == 5]
geojson_for_representation = geojson_for_representation.loc[geojson_for_representation['name']=='Pfaffenthal-Kirchberg', 'geojson'].values[0]

In [11]:
# We reconfigure the features for this selection
geojson_for_representation['features'][0]['properties'] = {"description": "travel time zone 5 mins",
                                      "fill": "#ff2500",
                                      "fill_opacity": 0.5}

In [12]:
from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure

In [13]:
# We transform our geojson structured code into geojson to be read by folium
geo_source = GeoJSONDataSource(geojson=json.dumps(geojson_for_representation))

In [14]:
import folium

map_Lux = folium.Map(location=[49.5259473,6.084417], width=1800, height=1800)

In [15]:
folium.GeoJson(json.dumps(geojson_for_representation), name='geojson').add_to(map_Lux)

In [16]:
map_Lux